### Importing Dependancies

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture

%pip install langchain langchain-community
%pip install langchainhub
%pip install langchain-chroma
%pip install langchain-groq
%pip install langchain-huggingface
%pip install gradio

In [3]:
!pip install -q -U transformers --upgrade bitsandbytes streamlit peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00


## Loading base model & LoRa Adapters from HuggingFace

In [13]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM , BitsAndBytesConfig
from IPython.display import Markdown, display
import torch

config = PeftConfig.from_pretrained("Koomemartin/unsloth-gemma2-9b-version3-100k")
base_model = AutoModelForCausalLM.from_pretrained("unsloth/gemma-2-9b-bnb-4bit")
model = PeftModel.from_pretrained(base_model, "Koomemartin/unsloth-gemma2-9b-version3-100k")

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Koomemartin/unsloth-gemma2-9b-version3-100k")

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [15]:
prompt_template='''
You are a math assistant. Answer the following math problem with a detailed, step-by-step solution. Be clear and concise in each step. If there are multiple approaches, select the most efficient method. Include any formulas or key concepts used, and provide the final answer at the end.

Instruction: {problem} \n
Response: {response}
'''

### Inference

In [ ]:
inputs = tokenizer(
[
    prompt_template.format(
        problem="Find the ordered pair  (𝑥,𝑦)that satisfies the equations  7𝑥−50𝑦=3and  3𝑦−𝑥=5", # instruction
        response="",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024)

<bos>
You are a math assistant. Answer the following math problem with a detailed, step-by-step solution. Be clear and concise in each step. If there are multiple approaches, select the most efficient method. Include any formulas or key concepts used, and provide the final answer at the end.

Instruction: Find the ordered pair  (𝑥,𝑦)that satisfies the equations  7𝑥−50𝑦=3and  3𝑦−𝑥=5 

Response: 


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


To find the ordered pair (x, y) that satisfies the equations 7x - 50y = 3 and 3y - x = 5, we can use the method of substitution.

Step 1: Solve the second equation for x.
3y - x = 5
x = 3y - 5

Step 2: Substitute the expression for x into the first equation.
7x - 50y = 3
7(3y - 5) - 50y = 3

Step 3: Simplify and solve for y.
21y - 35 - 50y = 3
-29y = 38
y = -38/29

Step 4: Substitute the value of y back into the expression for x.
x = 3y - 5
x = 3(-38/29) - 5
x = -114/29 - 145/29
x = -259/29

Step 5: Write the ordered pair (x, y).
(x, y) = (-259/29, -38/29)

Therefore, the ordered pair that satisfies the equations 7x - 50y = 3 and 3y - x = 5 is (-259/29, -38/29).<eos>


In [ ]:
inputs = tokenizer(
[
    prompt_template.format(
        problem="Sean adds up all the odd integers from 1 to 499, inclusive. Julie adds up all the integers from 1 to 300, inclusive. What is Sean's sum divided by Julie's sum?", # instruction
        response="",
    )
], return_tensors = "pt").to("cuda")

_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024)

<bos>
You are a math assistant. Answer the following math problem with a detailed, step-by-step solution. Be clear and concise in each step. If there are multiple approaches, select the most efficient method. Include any formulas or key concepts used, and provide the final answer at the end.

Instruction: Sean adds up all the odd integers from 1 to 499, inclusive. Julie adds up all the integers from 1 to 300, inclusive. What is Sean's sum divided by Julie's sum? 

Response: 
Sean's sum is the sum of all odd integers from 1 to 499, inclusive. We can use the formula for the sum of an arithmetic series to find Sean's sum:

Sum = (n/2) * (first term + last term)

In this case, n = 250 (since there are 250 odd integers from 1 to 499), the first term is 1, and the last term is 499. Plugging these values into the formula, we get:

Sean's sum = (250/2) * (1 + 499) = 125 * 500 = 62500

Julie's sum is the sum of all integers from 1 to 300, inclusive. We can use the same formula for the sum of an

In [ ]:
inputs = tokenizer(
[
    prompt_template.format(
        problem="Consider the stationary predictive model: 𝑋𝑡+𝑇̂=𝑎𝑋𝑡−1 + 𝑏𝑋𝑡−3 + 𝑐𝑍𝑡 where 𝑋𝑡+𝑇̂ denotes the estimated value of 𝑋𝑡+𝑇 at T time steps in the future. By minimising the expected mean square error between the predicted estimated value 𝑋𝑡+𝑇̂ and the observed value 𝑋𝑡+𝑇, Find the optimal values of a, b and c in terms of autocorrelation", # instruction
        response="",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024)

<bos>
You are a math assistant. Answer the following math problem with a detailed, step-by-step solution. Be clear and concise in each step. If there are multiple approaches, select the most efficient method. Include any formulas or key concepts used, and provide the final answer at the end.

Instruction: Consider the stationary predictive model: 𝑋𝑡+𝑇̂=𝑎𝑋𝑡−1 + 𝑏𝑋𝑡−3 + 𝑐𝑍𝑡 where 𝑋𝑡+𝑇̂ denotes the estimated value of 𝑋𝑡+𝑇 at T time steps in the future. By minimising the expected mean square error between the predicted estimated value 𝑋𝑡+𝑇̂ and the observed value 𝑋𝑡+𝑇, Find the optimal values of a, b and c in terms of autocorrelation 

Response: 
To find the optimal values of a, b, and c in terms of autocorrelation, we can use the following steps:

1. Define the mean square error (MSE) between the predicted estimated value 𝑋𝑡+𝑇̂ and the observed value 𝑋𝑡+𝑇:

MSE = E[(𝑋𝑡+𝑇̂ − 𝑋𝑡+𝑇)^2]

2. Substitute the stationary predictive model into the MSE expression:

MSE = E[(𝑎𝑋𝑡−1 + 𝑏𝑋𝑡−3 + 𝑐𝑍𝑡 − 𝑋𝑡+

## Model Pipeline to pass to the rag chain

In [16]:
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from IPython.display import Markdown, display

# Create a text-generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer, # Ensure pipeline runs on the selected device
    max_new_tokens=1024,
)

# Initialize the LangChain HuggingFacePipeline with the pipeline object
llm = HuggingFacePipeline(pipeline=pipe)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausa

In [17]:
prompt=prompt_template.format(problem="Sean adds up all the odd integers from 1 to 499, inclusive. Julie adds up all the integers from 1 to 300, inclusive. What is Sean's sum divided by Julie's sum?",
                              response="")


display(Markdown(llm(prompt)))

<ipython-input-17-c94f4b6170c0>:5: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  display(Markdown(llm(prompt)))



You are a math assistant. Answer the following math problem with a detailed, step-by-step solution. Be clear and concise in each step. If there are multiple approaches, select the most efficient method. Include any formulas or key concepts used, and provide the final answer at the end.

Instruction: Sean adds up all the odd integers from 1 to 499, inclusive. Julie adds up all the integers from 1 to 300, inclusive. What is Sean's sum divided by Julie's sum? 

Response: 
Sean's sum is the sum of all odd integers from 1 to 499, inclusive. We can use the formula for the sum of an arithmetic series to find Sean's sum:

Sum = (n/2) * (first term + last term)

In this case, n = 250 (since there are 250 odd integers from 1 to 499), the first term is 1, and the last term is 499. Plugging these values into the formula, we get:

Sean's sum = (250/2) * (1 + 499) = 125 * 500 = 62500

Julie's sum is the sum of all integers from 1 to 300, inclusive. We can use the same formula for the sum of an arithmetic series to find Julie's sum:

Sum = (n/2) * (first term + last term)

In this case, n = 300 (since there are 300 integers from 1 to 300), the first term is 1, and the last term is 300. Plugging these values into the formula, we get:

Julie's sum = (300/2) * (1 + 300) = 150 * 301 = 45150

Now, we need to find Sean's sum divided by Julie's sum:

Sean's sum divided by Julie's sum = 62500 / 45150

Simplifying the fraction, we get:

Sean's sum divided by Julie's sum = 1250 / 903

Therefore, Sean's sum divided by Julie's sum is approximately 1.385.

Final Answer: Sean's sum divided by Julie's sum is approximately 1.385.

In [ ]:
prompt=prompt_template.format(problem="Find the ordered pair  (𝑥,𝑦)that satisfies the equations  7𝑥−50𝑦=3 and  3𝑦−𝑥=5",
                              response="")
display(Markdown(llm(prompt)))


You are a math assistant. Answer the following math problem with a detailed, step-by-step solution. Be clear and concise in each step. If there are multiple approaches, select the most efficient method. Include any formulas or key concepts used, and provide the final answer at the end.

Instruction: Find the ordered pair  (𝑥,𝑦)that satisfies the equations  7𝑥−50𝑦=3 and  3𝑦−𝑥=5 

Response: 
To find the ordered pair (x, y) that satisfies the equations 7x - 50y = 3 and 3y - x = 5, we can use the method of substitution.

Step 1: Solve the second equation for x.
3y - x = 5
x = 3y - 5

Step 2: Substitute the expression for x into the first equation.
7x - 50y = 3
7(3y - 5) - 50y = 3

Step 3: Simplify and solve for y.
21y - 35 - 50y = 3
-29y = 38
y = -38/29

Step 4: Substitute the value of y back into the expression for x.
x = 3y - 5
x = 3(-38/29) - 5
x = -114/29 - 145/29
x = -259/29

Step 5: Write the ordered pair (x, y).
(x, y) = (-259/29, -38/29)

Therefore, the ordered pair that satisfies the equations 7x - 50y = 3 and 3y - x = 5 is (-259/29, -38/29).

## Initiating Embedding Model

In [18]:
from langchain_huggingface import HuggingFaceEmbeddings
# Initialize the embedding model
embed_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/114k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

## Loading and Preprocessing Retrieval Data
Dataset is from [Orca-math-word-problems](https://huggingface.co/datasets/microsoft/orca-math-word-problems-200k)

In [20]:
import pandas as pd
pd.read_csv('/content/drive/MyDrive/Chatbot/Orac_math_problems.csv').shape

(50000, 2)

In [22]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("/content/drive/MyDrive/Chatbot/", glob="**/*.csv", loader_cls=CSVLoader)

data = loader.load()

In [23]:
len(data) # size of the loaded data

172442

In [24]:
subset=data[:5000]  # subset of the chuncked documents

In [25]:
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

#split the loaded data into small chunks while overlap of 100 words
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
db_data = text_splitter.split_documents(subset)

In [26]:
# Set the batch size
batch_size = 96

# Calculate the number of batches
num_batches = len(db_data) // batch_size + (len(db_data) % batch_size > 0)

## Setting up VectorStore

In [27]:
# create a chorma database to index the chunked data
db = Chroma(embedding_function=embed_model , persist_directory="/content/drive/MyDrive/Vector DB")
retv = db.as_retriever(search_type="mmr", search_kwargs={'k': 2, 'fetch_k': 10})

# Iterate over batches
for batch_num in range(num_batches):
    # Calculate start and end indices for the current batch
    start_index = batch_num * batch_size
    end_index = (batch_num + 1) * batch_size
    # Extract documents for the current batch
    batch_documents = db_data[start_index:end_index]
    # add document to the retriever in batches
    retv.add_documents(batch_documents)
    print(start_index, end_index)


0 96
96 192
192 288
288 384
384 480
480 576
576 672
672 768
768 864
864 960
960 1056
1056 1152
1152 1248
1248 1344
1344 1440
1440 1536
1536 1632
1632 1728
1728 1824
1824 1920
1920 2016
2016 2112
2112 2208
2208 2304
2304 2400
2400 2496
2496 2592
2592 2688
2688 2784
2784 2880
2880 2976
2976 3072
3072 3168
3168 3264
3264 3360
3360 3456
3456 3552
3552 3648
3648 3744
3744 3840
3840 3936
3936 4032
4032 4128
4128 4224
4224 4320
4320 4416
4416 4512
4512 4608
4608 4704
4704 4800
4800 4896
4896 4992
4992 5088
5088 5184
5184 5280
5280 5376
5376 5472
5472 5568
5568 5664
5664 5760
5760 5856
5856 5952
5952 6048
6048 6144
6144 6240
6240 6336
6336 6432
6432 6528
6528 6624
6624 6720
6720 6816
6816 6912
6912 7008
7008 7104
7104 7200
7200 7296
7296 7392
7392 7488
7488 7584
7584 7680
7680 7776
7776 7872
7872 7968
7968 8064
8064 8160
8160 8256
8256 8352
8352 8448
8448 8544
8544 8640
8640 8736
8736 8832
8832 8928
8928 9024
9024 9120
9120 9216
9216 9312
9312 9408
9408 9504
9504 9600
9600 9696
9696 9792
9792 

In [28]:
query = " Stationary time series"
docs = db.similarity_search(query)
print(docs[0].page_content)

According to the problem, the store sold \( \frac{3}{5} \) of the T-shirts left from yesterday's sale this morning. So, the number of T-shirts sold this morning is \( \frac{3}{5}x \).

It is given that the number of T-shirts sold in the morning is equal to the number of T-shirts sold in the afternoon. Therefore, the number of T-shirts sold in the afternoon is also \( \frac{3}{5}x \).


## Building Components for RAG Chain

In [29]:
from langchain_core.prompts import PromptTemplate

rag_template = ("""
    You are a math assistant. Answer the following math problem with a detailed, step-by-step solution.
    Be clear and concise in each step. If there are multiple approaches, select the most efficient method.
    Include any formulas or key concepts used, and provide the final answer at the end.
    Use the provided context to answer the question. If the context is note relevant to the query ignore it.

    Context: {context}

    Question: {question}

    Answer:""")

rag_prompt = PromptTemplate.from_template(rag_template)

## Building the RAG Chain

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retv, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

## Testing the Chain



In [31]:
from IPython.display import display, Markdown

response = rag_chain.invoke("What is the sum of the first 100 prime odd numbers")
display(Markdown(response))  #context retrieved not relevant


    You are a math assistant. Answer the following math problem with a detailed, step-by-step solution.
    Be clear and concise in each step. If there are multiple approaches, select the most efficient method.
    Include any formulas or key concepts used, and provide the final answer at the end.
    Use the provided context to answer the question. If the context is note relevant to the query ignore it.

    Context: [Document(metadata={'row': 66, 'source': '/content/drive/MyDrive/Chatbot/Orac_math_problems.csv'}, page_content='Sum = n/2 * (first term + last term)\nSum = 100/2 * (1 + 199)\nSum = 50 * 200\nSum = 10000\n\nTherefore, the sum of the first 100 consecutive odd numbers starting from 1 is 10,000.'), Document(metadata={'row': 475, 'source': '/content/drive/MyDrive/Chatbot/Orac_math_problems.csv'}, page_content='So, the sum is:\n\nSum = 100/2 * (1 + 100)\nSum = 50 * 101\nSum = 5050\n\nTherefore, the sum of all numbers between 1 and 100 is 5050.')]

    Question: What is the sum of the first 100 prime odd numbers

    Answer:
    The sum of the first 100 prime odd numbers is 10000.
    ```
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    


In [32]:
query = "Find the ordered pair (𝑥,𝑦)that satisfies the equations 7𝑥−50𝑦=3 and  3𝑦−𝑥=5"

for chunk in rag_chain.stream(query):
            print(chunk, end="")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


 To find the ordered pair (𝑥,𝑦) that satisfies the equations 7𝑥−50𝑦=3 and  3𝑦−𝑥=5, we can use the method of substitution.

    From the second equation, we have:
    𝑥=3𝑦−5

    Substituting this value of 𝑥 into the first equation, we get:
    7(3𝑦−5)−50𝑦=3

    Simplifying the equation, we have:
    21𝑦−35−50𝑦=3
    −29𝑦−35=3

    Adding 35 to both sides, we get:
    −29𝑦=38

    Dividing both sides by -29, we get:
    𝑦=−3829

    Now, substituting this value of 𝑦 into the equation 𝑥=3𝑦−5, we get:
    𝑥=3(−3829)−5
    𝑥=−11429−5
    𝑥=−12929

    Therefore, the ordered pair (𝑥,𝑦) that satisfies the equations 7𝑥−50𝑦=3 and  3𝑦−𝑥=5 is:
    (𝑥,𝑦)=(−12929,−3829)

    Final Answer: The ordered pair (𝑥,𝑦) that satisfies the equations 7𝑥−50𝑦=3 and  3𝑦−𝑥=5 is (−12929,−3829).


## Gradio App

In [34]:
import gradio as gr

def rag_memory_stream(text):
    partial_text = ""
    for new_text in rag_chain.stream(text):
        partial_text += new_text
        # Yield an empty string to cleanup the message textbox and the updated conversation history
        yield partial_text


title = "Welcome to Mathlearn"
demo = gr.Interface(
    title=title,
    description='Provide mathematics problems to get step by step solutions',
    fn=rag_memory_stream,
    inputs="text",
    outputs="text",
    allow_flagging="never",
)

demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://150b35238b253b5aac.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [35]:
import gradio as gr

def rag_memory_stream(text):
    partial_text = ""
    for new_text in rag_chain.stream(text):
        partial_text += new_text
        # Yield an empty string to cleanup the message textbox and the updated conversation history
        yield partial_text


title = "Welcome to Mathlear"
demo = gr.Interface(
    title=title,
    description='Provide mathematics problems to get step by step solutions',
    fn=rag_memory_stream,
    inputs="text",
    outputs="text",
    # live=True,
    allow_flagging="never",
)

demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4cd05cea4a7e30bc6b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Streamlit Script

In [ ]:
from transformers import AutoModelForCausalLM , BitsAndBytesConfig , AutoTokenizer
from peft import PeftModel, PeftConfig

from time import time
from IPython.display import Markdown, display

import streamlit as st

class ModelInterface():

  def __init__(self):
      self.path_to_model = "unsloth/gemma-2-9b-bnb-4bit"
      self.max_new_tokens = 1024
      self.initialize_model()

  def initialize_model(self):
      start_time = time()
      self.tokenizer = AutoTokenizer.from_pretrained(self.path_to_model)
      tok_time = time()
      print(f"Load tokenizer: {round(tok_time-start_time, 1)} sec.")
      self.config = PeftConfig.from_pretrained("Koomemartin/unsloth-gemma2-9b-version3-100k")
      self.base_model = AutoModelForCausalLM.from_pretrained(
          self.path_to_model,

          low_cpu_mem_usage=True,
          device_map="cuda",
      )
      self.model = PeftModel.from_pretrained(self.base_model, "Koomemartin/unsloth-gemma2-9b-version3-100k",)
      mod_time = time()
      print(f"Load model: {round(mod_time-tok_time, 1)} sec.")


  def get_message_response(self, input_text):
      start_time = time()

      prompt_template='''
        <start_of_turn> user \n
        You are a math assistant. Answer the following math problem with a detailed, step-by-step solution. Be clear and concise in each step. If there are multiple approaches, select the most efficient method. Include any formulas or key concepts used, and provide the final answer at the end.

        Instruction: {problem} \n  <end_of_turn>  \n
        <start_of_turn> model \n
        Response:
           '''

      inputs=self.tokenizer(
        [
          prompt_template.format(problem=input_text)
        ] ,return_tensors='pt'
      ).to('cuda')


      outputs = self.model.generate(
              **inputs,
              max_new_tokens=self.max_new_tokens,
              eos_token_id=self.tokenizer.eos_token_id,
              )
      end_time = time()

      answer=self.tokenizer.decode(outputs[0])

      cleaned_response = answer.split("Response:")[1].strip()

      return cleaned_response




@st.cache_resource # if the model is loaded the cached version is used to avoid memory wastage
def load_model_interface():
    return ModelInterface()


model_interface = load_model_interface()

st.title('🦜🔗 Welcome to  MathLearn ')

# Streamlit session state to manage chat messages
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat history
for message in st.session_state.messages:
  with st.chat_message(message["role"]):
      st.markdown(message["content"])

# Accept user input and process response
if user_input := st.chat_input():
  st.session_state.messages.append({"role": "user", "content": user_input})
  with st.chat_message("user"):
      st.markdown(user_input)

  with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
          response_text = model_interface.get_message_response(user_input)
          st.write(response_text)

  # Save assistant's response to chat history
  st.session_state.messages.append({"role": "assistant", "content": response_text})



In [10]:
# import torch
# torch.cuda.empty_cache()

In [11]:
!wget -q -O - ipv4.icanhazip.com

34.16.205.55


In [12]:
! streamlit run '/content/drive/MyDrive/Chatbot/app (1).py' & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.2:8501
  External URL: http://34.16.205.55:8501

your url is: https://small-things-draw.loca.lt
2024-11-16 12:52:42.933995: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-16 12:52:42.957188: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-16 12:52:42.964367: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 12:52:42.982192: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instru